In [1]:
import tkinter
import random
import math

#file1 = open('JazzNetwork.dot', 'r+')
#file1 = open('LeagueNetwork.dot', 'r+')
file1 = open('LesMiserables.dot', 'r+')

def extractDot():
    global nodes
    nodes = []
    #edge values
    global edgv1
    edgv1 = {}
    global header 
    header = ""
    global footer 
    footer = ""
    count = 0
    #Parse line by line, check content, sort accordingly
    for i in file1:
        if len(i) == 1:
            continue
        else:
            i = i.strip()
            if '{' in i:
                header = i
            elif '}' in i:
                footer = i
            #These are edges with weights
            elif '=' in i and '-' in i:
                x = i.split("  ")
                edgv1[x[0]] = x[1].replace(";", "")
            #These are edges without weights
            elif '-' in i and '=' not in i:
                x = i.split(" ")
                u = x[2].split(";")
                if int(x[0]) in edgv1:
                    edgv1[int(x[0])].append(int(u[0]))
                else:
                    edgv1[int(x[0])] = [int(u[0])]
            #These are nodes
            else:
                x = i.split(";")
                nodes.append(int(x[0]))   



#get edges, check & remove duplicates/reverse edges
def getForceDirLO():
    global file
    file = open("GraphOutput.txt", "w+")
    global FDnodes 
    FDnodes = []
    for i in edgv1:
        if type(i) is str:
            tempo = []
            temporev = []
            a = i.split(" ")
            tempo.append(a[0])
            tempo.append(a[2])
            temporev.append(a[2])
            temporev.append(a[0])
            if tempo and temporev not in FDnodes:
                FDnodes.append(tempo)   
        elif type(i) is int:
            for h in edgv1[i]:
                tempo = []
                temporev = []
                tempo.append(str(i))
                tempo.append(str(h))
                temporev.append(str(h))
                temporev.append(str(i))
                if tempo and temporev not in FDnodes:
                    FDnodes.append(tempo)   

    file.write(str(len(nodes))+"\n")
    file.write(str(len(FDnodes))+"\n")
    for q in FDnodes:
        #print(q)
        file.write(q[0]+" ")
        file.write(q[1]+"\n")

        


x = extractDot()
#print(edgv1)
c = getForceDirLO()
#print(edgv1)
#print(FDnodes)

file.close()


In [2]:
graphname = 'GraphOutput'

#function to generate adjacency matrix.
def create_AdjM(nodesx, edges, l = []):
    V = nodesx
    E = edges
    adj_matrix = []
    for i in range(0, V):
        temp = []
        for j in range(0, V):
            temp.append(0)
        adj_matrix.append(temp)
    for j in range(2, (E * 2 + 2), 2):
        a = l[j]-1
        b = l[j+1]-1
        adj_matrix[a][b] = adj_matrix[b][a] = 0.1
    return adj_matrix


with open(graphname+".txt") as f:
    lines = f.read().split()

#create list out of input file
lines = [int(i) for i in lines]
#first line no of nodes second line no of edges
nodesx = lines[0]
#second line no of edges
edges = lines[1]


#call function to create adj matrix
d = create_AdjM(nodesx, edges, lines)

m = len(d)

#initialize TKinter interpreter & root window
root = tkinter.Tk()       
#create canvas widget for graph drawing
canvas = tkinter.Canvas(root, width=2000, height=2000, background="#FFFAFA")    
#.pack to amalgamate widgets into one 
canvas.pack()

#matrix of coordinates [[x,y]]
x = []
#matrix of forces working on each node
v = []
ids = []


#shift oval with graph movements
def move_oval(i):
    newx = int(x[i][0] * 660)  
    newy = int(x[i][1] * 660)  
    canvas.coords(ids[i], newx - 5, newy - 5, newx + 5, newy + 5)  


#canvas creation
for i in range(len(d)):
    xi = [random.random(), random.random()]
    x.append(xi)
    v.append([0.0, 0.0])
    id = canvas.create_oval(185, 185, 185, 185)

    ids.append(id)
    move_oval(i)

line_id = []

#plot coordinates for each node & all its edges
#id identifies edge, xi is node u & xj is node v
def line_Coord(id, xi, xj):
    canvas.coords(id,
                  int(xi[0] * 660),
                  int(xi[1] * 660),
                  int(xj[0] * 660),
                  int(xj[1] * 660))


for i in range(len(d)):
    for j in range(len(d)):
        if d[i][j] != 0:    
            id = canvas.create_line(0, 0, 0, 0)
            line_id.append(id)
            line_Coord(id, x[i], x[j])
            

#repulsive constant (i.e. magnetic repulsion) 
rep_c = 0.0008
#calculate repulsive force function
def repulse(xi, xj):  
    dx = xj[0] - xi[0]
    dy = xj[1] - xi[1]
    ds2 = dx * dx + dy * dy
    ds = math.sqrt(ds2)
    ds3 = ds2 * ds
    if ds3 == 0.0:
        const = 0
    else:
        const = rep_c / (ds2 * ds)
    return [-const * dx, -const * dy]


#spring constant (i.e. stiffness) 1 = stiff, 0 = not stiff
att_c = 1.0
#calculate attractive force function
def attract(xi, xj, dij): 
    dx = xj[0] - xi[0]
    dy = xj[1] - xi[1]
    ds = math.sqrt(dx * dx + dy * dy)
    dl = ds - dij
    const = att_c * dl / ds
    return [const * dx, const * dy]


#cooling factor: regulates energy dissipation, 1 and higher increases total eneregy per iteration
#between 0 and 1 reduces total energy per iteration
cool_f = 0.9
tempv = 2.099
tempz = 0.001
#function calculates movement for each nodes in relation to all other nodes
def move():
    force_temp = [0.0, 0.0]
    for i in range(len(d)):
        Fx = 0.0
        Fy = 0.0
        for j in range(len(d)):
            dij = d[i][j]
            Fij = 0.0
            #check if neighbour, if not calculate repulsive force
            if dij == 0.0:
                Fij = repulse(x[i], x[j])
            #else node is neighbour, calculate attractive force
            else:
                Fij = attract(x[i], x[j], dij)
                
            Fx += Fij[0]
            Fy += Fij[1]
        v[i][0] = (v[i][0] + tempz * Fx * tempv) * cool_f
        v[i][1] = (v[i][1] + tempz * Fy * tempv) * cool_f

        force_temp[0] = force_temp[0] + tempz * (v[i][0] * v[i][0])
        force_temp[1] = force_temp[1] + tempz * (v[i][1] * v[i][1])
        
        #total_en = math.sqrt(force_temp[0] * force_temp[0] + force_temp[1] * force_temp[1])
        #print("total energy contained in the graph: {}".format(total_en))


    for i in range(m):
        x[i][0] += v[i][0] * tempv
        x[i][1] += v[i][1] * tempv
        move_oval(i)

    
    tempq = 0
    for i in range(m):
        for j in range(m):
            if d[i][j] != 0:
                id = line_id[tempq]
                line_Coord(id, x[i], x[j])
                tempq += 1
                

    root.after(1, move)

root.after(1, move)

root.mainloop()
